In [1]:
import os
import pandas as pd
from dotenv import load_dotenv

os.environ["PATH"] += os.pathsep + "/Users/dursamiii/.nvm/versions/node/v22.20.0/bin"

In [2]:
load_dotenv()
token = os.getenv("x_auth_token")
print(token)

83bf74a1bebeca5e2dd585eb582dc906e4ab81ae


### Config

In [3]:
filename = "Hasil_Crawling.csv"
search_keyword = "Tepuk Sakinah"
limit = 100


### Mulai Nyari Tweet

In [4]:
# !npx -y tweet-harvest@latest -o "{filename}" -s "{search_keyword}" --tab LATEST -l {limit} --token {token}

### Lihat Hasil Crawl

In [5]:
file_path = f"tweets-data/{filename}"
df = pd.read_csv(file_path, delimiter=',')
display(df.head(5))

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1975873064484343928,Wed Oct 08 10:37:03 +0000 2025,46,udah tepuk sakinah https://t.co/504dDxPHQf,1975873064484343928,https://pbs.twimg.com/media/G2u2Eg0aMAAJpQu.jpg,NaN,in,NaN,0,0,5,https://x.com/undefined/status/197587306448434...,1863567571175546880,NaN
1,1973923616015462554,Fri Oct 03 01:30:38 +0000 2025,77,YANG UDAH NIKAH NGAKAK YANG BELOM NIKAH MUAAAK...,1973923616015462554,https://pbs.twimg.com/ext_tw_video_thumb/19739...,NaN,in,NaN,55,10,33,https://x.com/undefined/status/197392361601546...,1481089289035579392,NaN
2,1975184138199322949,Mon Oct 06 12:59:30 +0000 2025,160,tepuk sakinah tepuk trauma https://t.co/ihwpWX...,1975184138199322949,https://pbs.twimg.com/ext_tw_video_thumb/19751...,NaN,in,NaN,2,97,9,https://x.com/undefined/status/197518413819932...,761653636145356801,NaN
3,1975366930891719027,Tue Oct 07 01:05:51 +0000 2025,18,Cuma IQ di atas 80 yg bisa menangkap pesan mor...,1975366930891719027,https://pbs.twimg.com/ext_tw_video_thumb/19753...,NaN,in,NaN,7,12,7,https://x.com/undefined/status/197536693089171...,963751550,NaN
4,1974081204363432337,Fri Oct 03 11:56:50 +0000 2025,763,Menag Optimistis Tepuk Sakinah Bisa Redam Perc...,1974081204363432337,https://pbs.twimg.com/media/G2VYK0wbgAEu2pm.jpg,NaN,in,NaN,843,529,172,https://x.com/undefined/status/197408120436343...,23343960,NaN


## Mulai Preprocessing

### 1. Case Folding

teks dijadikan huruf kecil semua

In [6]:
df['casefolded'] = df['full_text'].str.lower()
df[['full_text','casefolded']].head(5)

,full_text,casefolded
0,udah tepuk sakinah https://t.co/504dDxPHQf,udah tepuk sakinah https://t.co/504ddxphqf
1,YANG UDAH NIKAH NGAKAK YANG BELOM NIKAH MUAAAK...,yang udah nikah ngakak yang belom nikah muaaak...
2,tepuk sakinah tepuk trauma https://t.co/ihwpWX...,tepuk sakinah tepuk trauma https://t.co/ihwpwx...
3,Cuma IQ di atas 80 yg bisa menangkap pesan mor...,cuma iq di atas 80 yg bisa menangkap pesan mor...
4,Menag Optimistis Tepuk Sakinah Bisa Redam Perc...,menag optimistis tepuk sakinah bisa redam perc...


### 2. Filtering

In [7]:
import re
import nltk
from nltk.corpus import stopwords

In [8]:
# nltk.download('stopwords')
id_words = set(stopwords.words('indonesian'))
eng_words = set(stopwords.words('english'))

# Tambahin daftar kata ga baku yg disingkat singkat
custom_stopwords = {
    'yg', 'sy', 'sdh', 'udh', 'dgn', 'dg', 'aja', 'nih', 'sih', 'sihh', 'nihh', 'ajaa', 'eh', 'ehh', 'ehhh',
    'dong', 'tp', 'gk', 'ga', 'nggak', 'knp', 'trs', 'aja', 'bgt', 'nya', 'rp', 'bgt', 'bgtt', 'bgttt', 'jgn',
    'bgtttt', 'sii', 'si', 'sih', 'siii', 'sihh', 'sihhh', 'jg', 'aj', 'tp', 'tpi', 'tuu', 'tuuu', 'tuhh', 'jgnn',
    'nya', 'nih', 'loh', 'kok', 'lah', 'deh', 'kan', 'ya', 'yaa', 'yaaa', 'kann', 'yh', 'yhh', 'yhhh', 'jg',
    'tuh', 'si', 'pun', 'pd', 'blm', 'kl', 'klo', 'klu', 'utk', 'biar','mah', 'deh', 'dehh', 'dehhh'
}

# Gabungkan dua set stopword
stop_words = id_words.union(eng_words)
stop_words = stop_words.union(custom_stopwords)

In [9]:
def clean_text(text):
    if not isinstance(text, str):
        return ""

    # Hapus mention (@user)
    text = re.sub(r'@\w+', '', text)
    
    # Hapus hashtag (#tag)
    text = re.sub(r'#\w+', '', text)
    
    # Hapus URL (http, https)
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Hapus angka
    text = re.sub(r'\d+', '', text)
    
    # Hapus simbol sm tanda baca, kecuali spasi
    text = re.sub(r'[^\w\s]', '', text)
    
    # Hapus spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Hapus stopword
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

# Terapkan ke kolom
df['filtered'] = df['casefolded'].apply(clean_text)

In [10]:
# Lihat hasilnya
df[['casefolded', 'filtered']].head(100)

,casefolded,filtered
0,udah tepuk sakinah https://t.co/504ddxphqf,udah tepuk sakinah
1,yang udah nikah ngakak yang belom nikah muaaak...,udah nikah ngakak belom nikah muaaak udah cera...
2,tepuk sakinah tepuk trauma https://t.co/ihwpwx...,tepuk sakinah tepuk trauma
3,cuma iq di atas 80 yg bisa menangkap pesan mor...,iq menangkap pesan moral lagu tepuk sakinah be...
4,menag optimistis tepuk sakinah bisa redam perc...,menag optimistis tepuk sakinah redam perceraia...
...,...,...
95,tepuk sakinah ini maksudnya baik tapi...,tepuk sakinah maksudnya
96,ald1! gatau kenapa tapi sender ngebayangin mem...,ald gatau sender ngebayangin member bikin tepu...
97,makasih tepuk sakinah udah menyatukan skynani,makasih tepuk sakinah udah menyatukan skynani
98,kalo ga pengen nikah tapi hafal tepuk sakinah...,kalo pengen nikah hafal tepuk sakinah gpp


### 3. Stemming

ubah jadi kata dasar semua

In [11]:
# nltk.download('stopwords')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemming langsung per kalimat
df['stemmed'] = df['filtered'].apply(lambda x: stemmer.stem(x))

In [12]:
# Lihat hasil
df[['filtered', 'stemmed']].tail(10)

,filtered,stemmed
151,tbh dont get ppl hating tepuk sakinahits clums...,tbh dont get ppl hating tepuk sakinahits clums...
152,pringses tuk tepuk sakinah sm abang shuaaa nie...,pringses tuk tepuk sakinah sm abang shuaaa nie...
153,cc kantor urusan agama tepuk sakinah direktora...,cc kantor urus agama tepuk sakinah direktorat ...
154,kalo selingkuh timggal panggil pake tepuk saki...,kalo selingkuh timggal panggil pake tepuk saki...
155,tweet au part red zone tepuk sakinah,tweet au part red zone tepuk sakinah
156,rumah tangga tepuk sakinah pajak tepuk coretax...,rumah tangga tepuk sakinah pajak tepuk coretax...
157,sli sljj,sli sljj
158,nikah karna hapal tepuk sakinah,nikah karna hapal tepuk sakinah
159,untung pas nikah tepuk sakinah bayangin suamik...,untung pas nikah tepuk sakinah bayangin suami ...
160,ahhhh cerai suamikuu kemarin udah tepuk sakina...,ahhhh cerai suamikuu kemarin udah tepuk sakina...


### 4. Tokenize

In [13]:
# nltk.download('punkt_tab')

In [14]:
from nltk.tokenize import word_tokenize

# Misal df['stemmed'] udah ada hasil stemming-nya
df['tokenized'] = df['stemmed'].apply(word_tokenize)

# Cek hasilnya
df[['stemmed', 'tokenized']].head(10)

,stemmed,tokenized
0,udah tepuk sakinah,"[udah, tepuk, sakinah]"
1,udah nikah ngakak bom nikah muaaak udah cerai ...,"[udah, nikah, ngakak, bom, nikah, muaaak, udah..."
2,tepuk sakinah tepuk trauma,"[tepuk, sakinah, tepuk, trauma]"
3,iq tangkap pesan moral lagu tepuk sakinah pasa...,"[iq, tangkap, pesan, moral, lagu, tepuk, sakin..."
4,menag optimistis tepuk sakinah redam cerai baca,"[menag, optimistis, tepuk, sakinah, redam, cer..."
5,boomer balas dendam genjet cipta tepuk sakinah,"[boomer, balas, dendam, genjet, cipta, tepuk, ..."
6,rumah tangga mulus turun sabar maaf janji suci...,"[rumah, tangga, mulus, turun, sabar, maaf, jan..."
7,tepuk sakinah doesnt make stay,"[tepuk, sakinah, doesnt, make, stay]"
8,one want tepuk sakinah,"[one, want, tepuk, sakinah]"
9,selamat hengky meyden bahagia sakinah tepuk sa...,"[selamat, hengky, meyden, bahagia, sakinah, te..."


### Export hasil preprocess

In [15]:
import pandas as pd

# Ubah ke tipe datetime
df['created_at'] = pd.to_datetime(df['created_at'], format='%a %b %d %H:%M:%S %z %Y')

# Ambil tanggal & jam
df['tanggal'] = df['created_at'].dt.date
df['jam'] = df['created_at'].dt.time

# Ambil kolom yang penting aja
df_export = df[['tanggal', 'jam', 'stemmed']]

# Ganti nama kolom biar rapi
df_export.columns = ['tanggal', 'jam', 'text']

# Export ke CSV
df_export.to_csv("hasil_preprocess_final.csv", index=False)